In [49]:
import openai
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
from dotenv import load_dotenv

import time
import plotly.express as px
import os
import pandas as pd
import numpy as np
from ast import literal_eval

In [2]:
load_dotenv()

True

In [3]:
openai.api_key = os.getenv("OPENAPI_KEY")

In [4]:
word_list = [
    "France",
    "Germany",
    "Aldi",
    "Lidl",
    "Cheese",
    "Porsche",
    "Cyberpunk",
    "Winnie the Poo",
    "John Cena",
    "Moon",
    "Programming",
    "REST API",
    "Feng Shui",
    "Qi",
    "Red",
    "Meth",
    "Methane",
    "Goodman",
    "Ellen Ripley",
    "Meat",
    "T800",
    "Stan Winston",
]

In [5]:
word_embeddings = []
for word in word_list:
    word_embeddings.append(get_embedding(word, engine="text-embedding-ada-002"))

In [6]:
# Make term and emebddings df
df = pd.DataFrame({"Word List": word_list, "Embeddings": word_embeddings})

In [9]:
# Export word embeddings to storage for later use. Saves API calls. 
# df.to_csv("word_embeddings.csv")

In [13]:
# df = pd.read_csv("word_embeddings.csv", index_col=[0])

In [46]:
df

,Word List,Embeddings,similarities
0,France,"[0.030798528343439102, -0.005776405334472656, ...",0.774784
1,Germany,"[0.01930953934788704, -0.026552394032478333, 0...",0.807754
2,Aldi,"[-0.007373601198196411, -0.004077282268553972,...",0.788313
3,Lidl,"[-0.003565473947674036, 0.0006513209082186222,...",0.784138
4,Cheese,"[0.01063595525920391, 0.0009781892877072096, 0...",0.776597
5,Porsche,"[0.005553688388317823, -0.003836322110146284, ...",0.792697
6,Cyberpunk,"[0.0036292874719947577, -0.017004676163196564,...",0.786766
7,Winnie the Poo,"[0.0006939491140656173, -0.022046230733394623,...",0.757211
8,John Cena,"[-0.013514889404177666, 0.004290861543267965, ...",0.760448
9,Moon,"[0.018854590132832527, -0.008488831110298634, ...",0.749311


In [48]:
df["Embeddings"][0]

[0.030798528343439102,
 -0.005776405334472656,
 -0.00395740894600749,
 -0.039648059755563736,
 0.0016248241299763322,
 0.012218542397022247,
 -0.014014002867043018,
 -0.023495646193623543,
 0.0034530663397163153,
 -0.023764628916978836,
 0.001746706897392869,
 0.009851493872702122,
 0.004293637350201607,
 -0.025109542533755302,
 0.0002464975113980472,
 0.004576069302856922,
 0.04863208532333374,
 -0.010362561792135239,
 0.026992421597242355,
 -0.020630978047847748,
 -0.0009952362161129713,
 0.012151297181844711,
 0.009320253506302834,
 0.01648864336311817,
 0.01569514535367489,
 0.004068364389240742,
 0.01004650630056858,
 -0.029077038168907166,
 0.00304622994735837,
 -0.006015127524733543,
 0.004831603262573481,
 -0.0009078168659470975,
 -0.023872220888733864,
 -0.04037431627511978,
 -0.00903109647333622,
 -0.021787604317069054,
 -0.010887078009545803,
 -0.01365087553858757,
 0.01554720476269722,
 -0.01588343270123005,
 0.021639663726091385,
 0.017147652804851532,
 0.00477108219638466

In [10]:
def embed(term):
    return get_embedding(term, engine="text-embedding-ada-002")


def check_similarity(search, compare):
    search_embedding = embed(search)
    compare_embedding = embed(compare)
    return cosine_similarity(search_embedding, compare_embedding)


def calculate_similarity(df, search_term, pprint=True):
    embedding = embed(search_term)
    df['similarities'] = df["Embeddings"].apply(lambda x: cosine_similarity(x, embedding))
    res = df.sort_values('similarities', ascending=False)
    res_20 = res.head(25)
    plotSimilarities(res_20, search_term, res_20.columns[0])
    return res_20


def plotSimilarities(df, search_term, hover_name):
    fig = px.scatter(
        df, 
        x="similarities", 
        y=[0]*len(df["similarities"]), 
        hover_name=hover_name,
        title="For Search Term:" + search_term    )
    #fig.update_layout(xaxis=dict(range=[0, 1]))
    fig.show()

In [12]:
feng_shui = "Das Layout deiner Wohnung lässt die Energie fließen. Das Sofa ist ideal in der Command Position platziert"
arnie = "Arnold Schwarzenegger"
t800 = "T-800"
wrestling = "Wrestling"
major_jaune = "Le major jaune"
rammstein = "Rammstein"

# This ONE

In [58]:
res = calculate_similarity(df, wrestling)

In [110]:
#######################################################

# Import Uwe Seeler Wikipedia Article

In [36]:
import re
import csv

# Open the input text file
with open("uwe_seeler_wikipedia.txt", "r") as f:
    text = f.read()

# Use regular expressions to split the text into sentences
sentences = re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", text)

# Open the output csv file
with open("uwe_seeler_wikipedia.csv", "w") as f:
    writer = csv.writer(f)

    # Write the header row
    writer.writerow(["Text"])

    # Initialize a variable to store the current sentence
    current_sentence = ""

    # Iterate over each sentence
    for sentence in sentences:
        # Check the length of the sentence
        if len(sentence.split()) < 5:
            # Append the current sentence with the next sentence
            current_sentence += sentence
        else:
            # Write the current sentence to the csv file
            writer.writerow([current_sentence + sentence])
            current_sentence = ""


In [14]:
# uweDF = pd.read_csv('uwe_seeler_wikipedia.csv')

In [15]:
def spread_api_calls(text):
    print(text)
    embedded_text = embed(text)
    time.sleep(1)
    return embedded_text

In [16]:
# uweDF["embeddings"] = uweDF["Text"].apply(lambda x: spread_api_calls(x))

In [17]:
# uweDF.rename(columns={'embeddings': 'Embeddings'}, inplace=True)

In [18]:
# Save for later use without using the API
# uweDF.to_csv("uwe_seeler_embeddings.csv")

In [50]:
uweDF = pd.read_csv("uwe_seeler_embeddings.csv", index_col=[0], converters={'Embeddings': pd.eval})

In [52]:
uweDF["Embeddings"]

0      [-0.008904125541448593, -0.013433169573545456,...
1      [-0.010335334576666355, -0.00362282688729465, ...
2      [-0.009197586216032505, 0.005072151776403189, ...
3      [-0.0018050437793135643, 0.005551062989979982,...
4      [-0.00017848526476882398, -0.00604895735159516...
                             ...                        
126    [0.01595756597816944, -0.0019251321209594607, ...
127    [0.018253158777952194, -0.014568915590643883, ...
128    [-0.006996626034379005, 0.013396690599620342, ...
129    [-0.005660532973706722, 0.014033057726919651, ...
130    [-0.0020005139522254467, -0.004108226392418146...
Name: Embeddings, Length: 131, dtype: object

# This One

In [53]:
res_20_df = calculate_similarity(uweDF, "Todestag")

In [56]:
res_20_df

,Text,Embeddings,similarities
130,August 2022 wurde er im engsten Familienkreis ...,"[-0.0020005139522254467, -0.004108226392418146...",0.813659
1,Juli 2022 in Norderstedt) war ein deutscher Fu...,"[-0.010335334576666355, -0.00362282688729465, ...",0.812661
121,Geburtstags den Uwe-Seeler-Preis.Seeler verzic...,"[-0.004903361666947603, -0.005271689500659704,...",0.806981
104,zwei Tage pro Woche aufwendete.,"[-0.004541965201497078, 0.009552310220897198, ...",0.805249
19,August 1954 zum zwischenzeitlichen 2:0 im Spie...,"[-0.022975454106926918, -0.0039054513908922672...",0.797823
17,In seinem ersten Pflichtspieleinsatz am 1.,"[-0.005907915066927671, -0.00102283398155123, ...",0.797484
119,\nAm 24.August 2005 wurde eine von dem Hamburg...,"[-0.018438395112752914, 0.006688946392387152, ...",0.796692
45,Im Mai 1972 trat „Uns Uwe“ mit einem Spiel des...,"[-0.0052361139096319675, -0.012654739432036877...",0.795916
129,a.ein Banner mit der Aufschrift Loyal und besc...,"[-0.005660532973706722, 0.014033057726919651, ...",0.793445
128,Juli 2022 starb Seeler im Alter von 85 Jahren ...,"[-0.006996626034379005, 0.013396690599620342, ...",0.792656
